Part 1

In [1]:
import numpy as np
import os, time

# ========== Data Loading ==========
def read_char(path):
    """Convert a text file to a 1D NumPy array of 0/1 pixels."""
    with open(path) as f:
        lines = [line.strip() for line in f if line.strip()]
    data = np.array([[1 if c == '#' else 0 for c in line] for line in lines])
    return data.flatten()

def load_dataset(folder):
    """Load all txt files and map A→+1, others→-1"""
    X, y = [], []
    for fname in os.listdir(folder):
        if not fname.endswith(".txt"): 
            continue
        label = fname[0].upper()
        target = 1 if label == 'A' else -1
        x = read_char(os.path.join(folder, fname))
        X.append(x)
        y.append(target)
    return np.array(X), np.array(y)

# ========== Perceptron Training ==========
def train_one_epoch(X, y, w, b, lr):
    errors = 0
    for xi, target in zip(X, y):
        z = np.dot(w, xi) + b
        y_pred = 1 if z >= 0 else -1
        if y_pred != target:
            w += lr * target * xi
            b += lr * target
            errors += 1
    return w, b, errors

def train_perceptron(X, y, w_init, b_init, lr=0.1, max_epochs=100):
    w = w_init.copy()
    b = b_init
    start = time.time()
    for epoch in range(1, max_epochs + 1):
        w, b, errors = train_one_epoch(X, y, w, b, lr)
        error_rate = errors / len(X)
        if errors == 0:
            elapsed = time.time() - start
            return w, b, error_rate, epoch, elapsed
    elapsed = time.time() - start
    return w, b, error_rate, max_epochs, elapsed

# ========== Initialization Strategies ==========
def init_weights(mode, n_in):
    if mode == "zeros":
        return np.zeros(n_in), 0.0
    elif mode == "uniform":
        return np.random.uniform(-0.5, 0.5, n_in), np.random.uniform(-0.5, 0.5)
    elif mode == "normal":
        return np.random.normal(0, 0.01, n_in), np.random.normal(0, 0.01)
    elif mode == "xavier":
        std = np.sqrt(2 / (n_in + 1))
        return np.random.normal(0, std, n_in), np.random.normal(0, std)
    elif mode == "kaiming":
        std = np.sqrt(2 / n_in)
        return np.random.normal(0, std, n_in), np.random.normal(0, std)
    else:
        raise ValueError("Unknown mode")

# ========== Main Experiment ==========
X_train, y_train = load_dataset("Characters-TrainSet")
X_test, y_test = load_dataset("Characters-TestSet")

results = []
for mode in ["zeros", "uniform", "normal", "xavier", "kaiming"]:
    print(f"\n--- Training with {mode} initialization ---")
    w0, b0 = init_weights(mode, X_train.shape[1])
    w, b, err, epochs, t = train_perceptron(X_train, y_train, w0, b0)
    y_pred = np.where(np.dot(X_test, w) + b >= 0, 1, -1)
    test_err = np.mean(y_pred != y_test)
    results.append((mode, err, test_err, epochs, t))
    print(f"Train error: {err*100:.2f}% | Test error: {test_err*100:.2f}% | Converged in {epochs} epochs | Time: {t:.4f}s")

# ========== Summary ==========
print("\nSummary Table:")
print("{:<10} {:>10} {:>12} {:>15} {:>12}".format("Init", "TrainErr%", "TestErr%", "EpochsToConv", "Time(s)"))
for mode, err, test_err, epochs, t in results:
    print(f"{mode:<10} {err*100:>10.2f} {test_err*100:>12.2f} {epochs:>15} {t:>12.4f}")



--- Training with zeros initialization ---
Train error: 0.00% | Test error: 4.76% | Converged in 3 epochs | Time: 0.0039s

--- Training with uniform initialization ---
Train error: 0.00% | Test error: 14.29% | Converged in 4 epochs | Time: 0.0000s

--- Training with normal initialization ---
Train error: 0.00% | Test error: 0.00% | Converged in 3 epochs | Time: 0.0000s

--- Training with xavier initialization ---
Train error: 0.00% | Test error: 4.76% | Converged in 5 epochs | Time: 0.0000s

--- Training with kaiming initialization ---
Train error: 0.00% | Test error: 0.00% | Converged in 3 epochs | Time: 0.0000s

Summary Table:
Init        TrainErr%     TestErr%    EpochsToConv      Time(s)
zeros            0.00         4.76               3       0.0039
uniform          0.00        14.29               4       0.0000
normal           0.00         0.00               3       0.0000
xavier           0.00         4.76               5       0.0000
kaiming          0.00         0.00        